
# 헬스케어 분석 프로젝트 — 목표 & 세부 과제

**데이터:** `healthcare_dataset.csv`

**공통 대원칙**
- **입원 시점(또는 직전)에 이용 가능**한 정보만으로 예측/비교를 수행합니다.
- 어떤 통계검정/모델을 사용할지는 **알아서 결정**합니다. 다만, 데이터 타입/그룹 수/분포 가정/질문의 성격을 근거로 선택 이유를 명확히 서술하세요.
- p값만 보고 결론 내리지 말고, **효과크기/불확실성(신뢰구간 등)**과 **운영적 의미**를 함께 제시하세요.


## 0) 데이터 적재

In [1]:

import pandas as pd
from pathlib import Path

DATA_PATH = Path("./data/healthcare_dataset.csv")
df = pd.read_csv(DATA_PATH)
df.head()


,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal



## 1) 데이터 인테이크 & 위생
**할 일**
- 컬럼 타입 확인, 날짜 파싱(가능한 경우), 결측/중복/논리 오류 점검

**제출물**
- 열별 결측률 표
- 발견된 이슈 및 처리 계획(간단 서술)
- **입원 시점에 사용 가능한 컬럼** 목록 vs **불가 컬럼** 목록

In [ ]:

# 결측률 표



,결측비율
Name,0.0
Age,0.0
Gender,0.0
Blood Type,0.0
Medical Condition,0.0
Date of Admission,0.0
Doctor,0.0
Hospital,0.0
Insurance Provider,0.0
Billing Amount,0.0


In [ ]:

# 날짜형 파싱

/tmp/ipykernel_1449994/1797131018.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors="coerce")


Name                          object
Age                            int64
Gender                        object
Blood Type                    object
Medical Condition             object
Date of Admission     datetime64[ns]
Doctor                        object
Hospital                      object
Insurance Provider            object
Billing Amount               float64
Room Number                    int64
Admission Type        datetime64[ns]
Discharge Date        datetime64[ns]
Medication                    object
Test Results                  object
dtype: object


### 의사결정 체크포인트
- 여러분의 **주요 결과변수**는 무엇인가요? (범주형/연속형, 수준 수 명시)
- 입원 시점 기준으로 **사용 가능한 컬럼만** 모아 `df_admit`을 구성하세요.
- 파생변수를 만들 경우, 입원 시점 기준에서 **허용/금지** 여부의 근거를 적으세요.



---
# 트랙 선택 (A / B / C 중 1개 이상 선택)

- **트랙 A**: 검사결과(정상/비정상/미결정) 인사이트 & 분류  
- **트랙 B**: 청구금액(Billing Amount) 인사이트 & 회귀  
- **트랙 C**: 입원기간(LOS) 인사이트 & 예측

필요하면 2개 이상 수행해도 됩니다.



## 트랙 A — 검사결과 인사이트 & 분류
**큰 목표:** 입원 시점 정보로 **비정상 검사** 위험 신호를 조기에 포착할 수 있는 규칙을 도출하고, 간단한 분류 프로토타입으로 **알람 임계값**을 제안한다.



### A1. 현황 파악
- `Test Results` 분포를 표와 그래프로 제시하고, “미결정”의 운영상 리스크를 2–3문장으로 설명하세요.



### A2. 내원 유형 차이
- `Admission Type`(응급/선택/긴급) **별 비정상 비율이 동일한가?**
- 적절한 방법으로 **차이 유무**를 평가하고, **효과크기**로 어느 쪽이 높은지 정량화하세요.



### A3. 성별/연령대 패턴
- `Gender`, 직접 정의한 `AgeBand`에 따라 **비정상 비율의 차이**가 있는지 평가하세요.
- “연령 증가에 따라 위험이 단조롭게 변하는가?”도 함께 답하세요.



### A4. 진단명 영향
- 상위 진단군 5개 + “기타”를 구성하여 **각 군의 비정상 비율**을 비교하고,
- **가장 높은 군과 기준군**의 차이를 정량적으로 보고하세요.



### A5. 예측 가능성 탐색
- 입원 시점 변수만으로 **기준선 분류기**와 **개선 분류기**를 각각 구축하세요. (모델 종류 자유)
- 검증 방식, **두 가지 지표**, **혼동행렬**을 제시하고,
- **중요한 변수 조합/상호작용**을 근거로 설명하세요.



### A6. 알람 임계값 제안
- “비정상 확률 ≥ τ” 형태로 **τ=3개(낮음/중간/높음)** 시나리오를 제시하세요.
- 알람 건수·재현율·정밀도를 표로 비교하고, **권장 τ** 1개를 선택해 근거를 쓰세요.



### A7. 형평성/신뢰성 점검
- 성별 또는 고령(예: 65+) 서브그룹에서 **성능·보정(calibration)**을 비교하고,
- 편향 징후가 있으면 **완화 방안**을 제안하세요.



### A8. 요약 제안
- “응급 + 특정 진단 + 고령” 등 **현장 규칙 2–3개**를 문장으로 정리하고,
- 잘못 분류 시의 **리스크**도 함께 기술하세요.



## 트랙 B — 청구금액 인사이트 & 회귀
**큰 목표:** 입원 시점 정보로 **청구금액의 주요 결정요인**을 밝히고, 과소/과다 청구 위험을 줄이는 **간단한 예측 도구**를 만든다.



### B1. 분포·이상값 이해
- `Billing Amount` 분포를 제시하고, **극단값 처리 규칙**을 스스로 정해 근거를 설명하세요.



### B2. 보험자/내원유형 차이
- `Insurance Provider`와 `Admission Type`에 따라 **평균 금액이 같은지/다른지** 평가하고,
- **얼마나** 다른지 효과크기와 함께 제시하세요.



### B3. 체류기간(LOS)와의 관계
- `LOS_days`(직접 생성)와 금액의 **관계 형태**(선형/비선형/단조)를 진단하고,
- 관계 강도를 **두 방식 이상**으로 요약하세요.



### B4. 다변량 관점
- (입원 시점 변수 중심) 금액에 영향을 주는 **상위 3개 요인**을 찾아 근거를 들어 설명하세요.



### B5. 예측 프로토타입
- 기준선 회귀 vs 개선 회귀(모델 자유), 검증 방식과 **오차지표 3개**(예: RMSE/MAE/MAPE) 및 **잔차 진단**을 제시하세요.



### B6. 고비용 플래그 규칙
- 상위 x% 고비용 환자를 조기 식별하기 위한 **간단 규칙**을 제안하고,
- **재현율/정밀도/알람 건수** 트레이드오프를 표로 비교하세요.



### B7. 서브그룹 일관성
- 보험자 A vs B(택1) 또는 연령대별로 **예측오차의 편향**이 있는지 점검하고, 완화 방안을 1가지 제시하세요.



### B8. 요약 제안
- “입원 시 **이 조건이면** 고비용 가능성 ↑, **이 조치는** 비용 ↓” 형태로 **운영 가이드 2–3개**를 작성하세요.



## 트랙 C — 입원기간(LOS) 인사이트 & 예측
**큰 목표:** 입원 시점 정보로 **장기입원 위험 요인**을 규명하고, 병상회전을 개선할 **예측/분류 규칙**을 제안한다.



### C1. LOS 생성·품질 점검
- `LOS_days = (Discharge Date − Date of Admission)` 생성 후, **음수/0/이상치**를 점검하고 처리 방안을 제시하세요.



### C2. 내원유형/진단별 차이
- 내원유형·상위 진단군별 **평균 LOS 차이**가 있는지 평가하고, **실무적 의미**를 효과크기로 설명하세요.



### C3. 연령/성별 패턴
- 연령대·성별에 따른 **LOS의 경향**(단조/상호작용)을 시각·수치로 요약하세요.



### C4. 자원 연계 지표
- LOS가 길수록 `Billing Amount`도 커지는지 **관계 강도**를 진단하고,
- “LOS 기반 자원 계획” 시사점을 2–3문장으로 작성하세요.



### C5. 예측 프로토타입(회귀 또는 이진 분류)
- 회귀(연속 LOS) 또는 분류(상위 20% 장기입원 vs 나머지) 중 하나를 택해,
- 기준선 vs 개선 모델, 검증·지표, **오류/혼동 분석**을 제시하세요.



### C6. 병상 운영 임계값
- “장기입원 확률 ≥ τ”의 τ를 2–3개 제안하고,
- 실제 병상 운영 관점에서 적정 임계값을 선택해 근거를 쓰세요.



### C7. 형평성 점검
- 서브그룹(예: 고령/저연령)별 **성능·캘리브레이션**을 비교하고, 개선안을 제시하세요.



### C8. 요약 제안
- “이런 케이스는 장기입원 위험이 높으므로 **사전 개입** 필요” 등 **정책 제안 2–3개**를 작성하세요.



---
## 제출 형식(공통)
- **리포트**(md/pdf): 각 문항에 대해 질문의 **의미**, 데이터·가정 확인, 선택한 방법의 **근거**, 수치(효과크기/불확실성/지표), **운영적 해석** 포함
- **코드**: 전처리→탐색→비교→예측→형평성 순으로 재현 가능하게 구성
- **그림/표**: 지표표, 혼동/잔차, 서브그룹 비교, 임계값(τ) 시나리오 표

## 채점 포인트(요약)
- **질문 이해 & 방법 선택 근거**
- **효과크기·불확실성·가정 점검**
- **누출 방지 & 검증 설계**
- **운영적 시사점(정책/임계값/규칙의 현실성)**
